In [14]:
from elasticsearch import Elasticsearch


es = Elasticsearch(
    "https://localhost:9200",
    verify_certs=False,
    ssl_show_warn=False,
    basic_auth=("elastic", "elastic") 
)

try:
    info = es.info()
    print("✅ Connected to Elasticsearch!")
    print(f"Cluster name: {info['cluster_name']}")
    print(f"Version: {info['version']['number']}")
except Exception as e:
    print("❌ Connection failed:", e)

✅ Connected to Elasticsearch!
Cluster name: elasticsearch
Version: 8.5.1


In [ ]:
source_index = "trump_comments_scored"
target_index = "search_all"



reindex_body = {
    "source": {
        "index": source_index
    },
    "dest": {
        "index": target_index
    },
    "script": {
        "source": """
            if (!ctx._source.containsKey('bertweet_sentiment')) {
                ctx.op = "noop";
                return;
            }
           
            if (ctx._source.containsKey('id')) {
                ctx._source.id = ctx._source.id.toString();
                ctx._id = ctx._source.id;
            }

            if (ctx._source.containsKey('type')) {
                ctx._source.type = ctx._source.type.toString();
                if (ctx._source.type == 'comments') {
                    ctx._source.type = 'comment';
                }
            }

            if (ctx._source.containsKey('platform')) {
                ctx._source.platform = ctx._source.platform.toString();
            }
            if (!ctx._source.containsKey('platform')) {
                ctx._source.platform = 'mastodon';
            }



            if (ctx._source.containsKey('bertweet_sentiment')) {
                ctx._source.sentiment_score = ctx._source.remove('bertweet_sentiment');
            }

            if (ctx._source.containsKey('post_id')) {
                def pid = ctx._source.post_id.toString();
                def platform = ctx._source.containsKey('platform') ? ctx._source.platform : null;
                if (platform != null && !pid.startsWith(platform + "_post_")) {
                    ctx._source.post_id = platform + "_post_" + pid;
                } else {
                    ctx._source.post_id = pid;
                }
            }
        """
    }
}


try:
    response = es.reindex(
        body=reindex_body,
        wait_for_completion=True,
        refresh=True,
        request_timeout=600
    )
    print("✅ Reindex completed.")
    print(response)
except Exception as e:
    print("❌ Reindex failed:", e)


/var/folders/jj/5cvzryh12_z87_3sbb0x4wpm0000gn/T/ipykernel_21084/1111559791.py:51: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  response = es.reindex(


✅ Reindex completed.
{'took': 2836, 'timed_out': False, 'total': 8828, 'updated': 8828, 'created': 0, 'deleted': 0, 'batches': 9, 'version_conflicts': 0, 'noops': 0, 'retries': {'bulk': 0, 'search': 0}, 'throttled_millis': 0, 'requests_per_second': -1.0, 'throttled_until_millis': 0, 'failures': []}
